In this notebook, we generate Erdős–Rényi graphs, draw random subsets and check if they resolve the graph. When we find a subset size which resolve the graph around half of the time, we use this size as a proxy for the metric dimension. We do those simulations for various values of $p$ but a fixed size of the graph $n$ and compare it to theoretical results.

In [ ]:
import networkx as nx
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import sys
sys.path.append('../')
from helpers import *
import plotly.graph_objs as go

In [ ]:
p_range = np.linspace(0.1, 0.9, 20)
n = 300
solutions = {}
nb_of_iters = 100

for p in tqdm(p_range):
    
    G = nx.erdos_renyi_graph(n, p)
    length = dict(nx.all_pairs_shortest_path_length(G))
    node_list = list(G.nodes())
    nb_of_iters = 100
    
    high = n
    low = 0
    
    # Binary search to find the "transition value"
    while low + 1 < high:
        middle = (high+low) // 2
        num_nodes = middle # Number of nodes to sample
        node_list = list(G.nodes())
        count = 0
        for i in range(nb_of_iters):
            nodes = set(random.sample(node_list, num_nodes)) # Random set of nodes to test
            if is_resolving_set(G, nodes, length):
                count += 1
        v = count / nb_of_iters
        if (count / nb_of_iters) > 0.5:
            high = middle
            v_high = v 
        else:
            low = middle
            v_low = v
    if abs(v_low - 0.5) < abs(v_high - 0.5):
        solutions[p] = low
    else:
        solutions[p] = high

In [ ]:
# Define your data
x = list(solutions.keys())
y = list(solutions.values())

x_t = p_range
y_t = [tvalue_case_1_2_p_const(n, pi) for pi in x_t]

# Define the trace for the scatter plot
trace = go.Scatter(x=x, y=y, mode='markers+lines', name='Simulations with n={} and p={}'.format(n, p))
trace_t = go.Scatter(x=x_t, y=y_t, mode='lines', name='Simulations with n={} and p={}'.format(n, p))

# Define the layout
layout = go.Layout(title='Probability of resolving the graph as a function of the subset cardinality', 
                   title_x=0.5,
                   xaxis=dict(title='Cardinality of the subset'), 
                   yaxis=dict(title='Probability of resolving the graph'),
                   legend=dict(x=0.67, y=0.08, orientation='v'))

# Combine the traces and layout into a figure
fig = go.Figure(data=[trace, trace_t], layout=layout)

# Show the figure
fig.show()